## ¿Hay asignaturas "outliers" que tienen notas de los alumnos muy diferentes del resto?

#### Con este planteamiento queremos verificar si hay asignaturas en las cuales los alumnos tienen notas muy diferentes respecto a las demás asignaturas o si las notas de alguna asignatura tienen una baja correlación en comparación con las notas de las demás asignaturas.

In [1]:
import numpy as np
import pandas as pd
import json

#####Función que permite selecionar un subconjunto que cumpla con una determinada condición de un dataFrame

In [2]:
def select_rows(df, group, col):
    #Creamos un nueva DataFrame
    out = pd.DataFrame()
    tmp = df.copy()
    # Recorremos los grupos
    for s in group:
        # Añadimos al DataFrame el nuevo elemento
        out = pd.concat([out, tmp[tmp[col] == s]])
    out = out.reset_index(drop=True)
    return out

##### Función que permite selecionar las notas de sus alumnos es función de unas asignaturas dadas

In [3]:
def table_students(df, group_subjects):
    # Con select_rows_by_subjects , nos quedamos con sólo las calificaciones de las asignaturas de primer curso
    grp = select_rows(df, group_subjects, 'id_assig')
    grp = grp[grp["tipus_apunt"] != "Convalidat"]
    # Agrupamos las asignaturas por el NIUB
    students_by_niub = grp.groupby('id_alumne').size()
    # De esta agrupación nos quedamos con los NIUBS que aparecen len(group_subjects) veces.
    students_by_niub = students_by_niub.index[students_by_niub >= len(group_subjects)]
    # Creamos una tabla tal que: alumnos-assignaturas y a cada celula la nota correspondiente
    grp_qual = grp.pivot_table('nota_primera_conv', index='id_alumne',columns='id_assig', aggfunc='max')
    # Ahora que sabemos los alumnso que han cursado  asignatueas deseadas, los selecionamos.
    grp_qual = grp_qual.ix[students_by_niub]
    grp_qual.dropna(inplace=True, axis=0)
    return grp_qual

##### Cargamos las tablas con los datos de las calificaciones y de las asignaturas

In [4]:
assig = pd.read_csv("datos3/assigs.csv", na_values=np.nan)
qualifications = pd.read_csv("datos3/qualifications.csv", na_values=np.nan)

##### Nos quedamos con las asignaturas del grado de Ingeniería Informática del primer, segundo y tercer curso

In [5]:
# Eliminamos las asignaturas que no sean del grado de Ingeniería Informática
assig2 =assig[assig['id_enseny_assig']  == 'G1077']
# Selecionamos las asignaturas del primer, segundo y tercer curso
assig_ter = select_rows(assig2, [1,2,3], 'curs_assig')['id_assig']
# Pasamos los identificadores de las asignaturas a una lista
list_assig = assig_ter.values

##### Selecionamos las asignaturas del cursto curso y las mostramos

In [6]:
assig_cuart = select_rows(assig2, [4], 'curs_assig')[['id_assig', 'desc_assig']]
print assig_cuart

    id_assig                                 desc_assig
0     364313                         Ètica i Legislació
1     364316                    Enginyeria del Software
2     364319                                   Robòtica
3     364320  Aprenentatge Automàtic i Mineria de Dades
4     364321      Intel·ligència Artificial Distribuïda
5     364324                     Processament d'Imatges
6     364325                     Tecnologies Multimèdia
7     364326                Computació Orientada al Web
8     364327                   Bases de Dades Avançades
9     364329                        Software Concurrent
10    364318                 Pràctiques en Empreses III
11    364318                 Pràctiques en Empreses III
12    364331                   Pràctiques en Empreses I
13    364331                   Pràctiques en Empreses I
14    364332                  Pràctiques en Empreses II
15    364332                  Pràctiques en Empreses II
16    364317                      Treball de Fi 

##### Añadimos a la lista de asignaturas, las 3 asignaturas obligatorias del cursto curso: Ètica i Legislació, Enginyeria del Software y Treball de Fi de Grau

In [7]:
list_assig = list(list_assig)

list_assig.append(364313)
list_assig.append(364316)
list_assig.append(364317)

##### Nos quedamos con las notas de las asignaturas mas arriba selecionadas. Tendremos en cada fila, el identificador del alumno junto con las notas de este alumno en cada uno de las asignaturas

In [8]:
# Obtenemos una tabla con las calificaciones de los alumnos por cada asignatura
tt = table_students(qualifications, list_assig)
# Hacemos la traspuesta de la matriz
ttt = tt.T

##### Pasamos la matriz con las calificaciones a un DataFrame y obtenemos la matriz de correlación de Spearman entre las calificaciones de cada asignatura

In [9]:
coll = []
coll.append(list_assig[0])
# Creamos un nuevo DataFrame para guardar los datos
df1 = pd.DataFrame(ttt.ix[list_assig[0]], columns=coll)
# Añadimos las calificaciones de cada asignatura al DataFrame
for i in range(1,len(list_assig)):
    df1[list_assig[i]] = pd.Series(ttt.ix[list_assig[i]], index=df1.index)
# Obtenemos la matriz de correlación
corr= df1.corr()
ddd = corr.as_matrix()

##### A continuaciñon creamos un JSON tal que por cada asignatura se selecionan las 4 asignaturas mas correlacionadas. En el caso de que todos los coeficientes de correlación de una asignaturas son menores que 0.5, esta asignaturas se queda sin relaciones, es decir outlier.

In [10]:
jsonn = []

# Recorremos la lista de las asignaturas
for asi in list_assig:
    # Obtenemos la lista de correlaciones de la asignatura ascual
    yy = corr.ix[asi]
    # Ordenamos las asignaturas en función de sus correlaciones
    yy_sort = yy.order(ascending=False)[1:5]
    indx_values =  yy_sort.values
    indx = yy_sort.index.values
    leg = []
    # Recorremos la lista de las asignaturas correlacionadas y nos quedamos con las mejores 4
    k = 0
    for ind in indx:
        # Verificamos si el indice de correlación supero el limite de 0.5
        if indx_values[0] > 0.5:
            # Creamos el string del JSON
            assig_s = "a." + assig[assig['id_assig'] == ind]['desc_assig'].values[0]
            leg.append(assig_s)
        k+=1
        
    assig_p = "a." + assig[assig['id_assig'] == asi]['desc_assig'].values[0]
    d = {} 
    # Añadimos los nuevos elementos al JSON
    d["name"] = assig_p
    d["size"] = 5000
    d["imports"] = leg
    jsonn.append(d)

##### Guardamos el JSON en un archivo al disco

In [11]:
with open('readme-flare-imports.json', 'w') as outfile:
    json.dump(jsonn, outfile)
    
#print json.dumps(jsonn)